<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: Approximation
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.optimize

import numerics.change_fonts
numerics.change_fonts.change_fontsize()

## Load and plot test data

The test data set are **beach lines** from Richmond Gulf, an area along the shores of Hudson Bay in Canada.

We load the data set with the `csv` package:

In [ ]:
csvfile = open('data/fit_richmondgulf.in', newline='')
beachlines = csv.reader(csvfile)
next(beachlines)
i=0
tobs=np.array([])
terr=np.array([])
hobs=np.array([])
herr=np.array([])
for row in beachlines:
    #print (row[0].split())
    tobs = np.append(tobs,float(row[0].split()[0]))
    terr = np.append(terr,float(row[0].split()[1]))
    hobs = np.append(hobs,float(row[0].split()[2]))
    herr = np.append(herr,float(row[0].split()[3]))
    i=i+1

----
## Fit with non-linear functions

text

In [ ]:
def f_exp(a,x):
    y = a[0] * (np.exp(x/a[1])-1.)
    return y

def f_exp_residual(a,x,y):
    y = a[0] * (np.exp(x/a[1])-1.) - y
    return y

In [ ]:
coeff = scipy.optimize.least_squares(f_exp_residual, [100,8000], args=(tobs,hobs), loss='cauchy')
#coeff = scipy.optimize.least_squares(f_exp_residual, [100,8000], args=(tobs,hobs), method='lm')
print(coeff.x)

n = len(tobs)
hpred = np.zeros(n)
for i in range(n):
    hpred[i] = f_exp(coeff.x,tobs[i])

chi2 = 0.
for i in range(n):
    chi2 = chi2 + ((hobs[i]-hpred[i])**2 / herr[i]**2)
print(chi2)
mu=1.e10
year2sec=365.25*24.*3600.
eta = coeff.x[1]*year2sec*mu
print('Viscosity [Pas]: ',eta)

In [ ]:
plt.figure(figsize=(10,6))
plt.title('Richmond Gulf, Hudson Bay, Canada')
plt.xlim([10000,0])
plt.ylim([0,320])
plt.xlabel('Time [a BP]')
plt.ylabel('Relative sea level [m]')
plt.errorbar(tobs,hobs,xerr=terr,yerr=herr,linewidth=0,elinewidth=3,ecolor='black')
plt.plot(tobs,hobs,linewidth=0,marker='o',markersize=10,label='data')
plt.plot(tobs,hpred,linewidth=2,color='red',label='non-linear function fit')
#plt.plot(tobs,ypred,label='scipy')
plt.text(8000.,300.,'$\chi^2$='+str(round(chi2,2)))
plt.legend()

----
[next>](lib08_optimisation.ipynb)